In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
from torchvision.models import resnet18

In [3]:
model = resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 2)
model = model.to(device)

epochs = 2
learning_rate = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
creterion = nn.CrossEntropyLoss()

d:\Anconda\envs\DL\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anconda\envs\DL\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
import gen_train_test_data
trainloader = gen_train_test_data.get_trainloader('data_train/')
testloader = gen_train_test_data.get_testloader('data_test/')

losses = []
train_accuracies = []
train_precisions = []
train_recalls = []
train_F1_scores = []
test_accuracies = []
test_precisions = []
test_recalls = []
test_F1_scores = []
turns = c1 = c2 = 0
tp = tn = fp = fn = 0

for epoch in range(epochs):
    model = model.train()
    for x, y in trainloader:
        inputs = x.to(device)
        labels = y.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = creterion(outputs, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        with torch.no_grad():
            if (c1%100 == 0 and c1 != 0):
                accuracy = (tp+tn)/(tp+tn+fp+fn)
                precision = tp/(tp+fp) if ((tp+fp)!=0) else 0
                recall = tp/(tp+fn) if ((tp+fn)!=0) else 0
                train_accuracies.append(accuracy)
                train_precisions.append(precision)
                train_recalls.append(recall)
                train_F1_scores.append((2*precision*recall/(precision+recall)) if ((precision+recall)!=0) else 0)
                tp = tn = fp = fn = 0
            turns += 1
            c1 += 10
            _, predicted = torch.max(outputs.cpu(), dim=1)
            labels = labels.cpu()
            tp += torch.sum((labels.reshape(-1) == 0) & (predicted == 0)).item()
            tn += torch.sum((labels.reshape(-1) == 1) & (predicted == 1)).item()
            fp += torch.sum((labels.reshape(-1) == 1) & (predicted == 0)).item()
            fn += torch.sum((labels.reshape(-1) == 0) & (predicted == 1)).item()
            print(f'turns:{turns}, loss:{loss.item()}')

    model = model.eval()
    with torch.no_grad():
        tp = tn = fp = fn = 0
        for inputs, labels in testloader:
            if (c2%100 == 0 and c2 != 0):
                accuracy = (tp+tn)/(tp+tn+fp+fn)
                precision = tp/(tp+fp) if ((tp+fp)!=0) else 0
                recall = tp/(tp+fn) if ((tp+fn)!=0) else 0
                test_accuracies.append(accuracy)
                test_precisions.append(precision)
                test_recalls.append(recall)
                test_F1_scores.append((2*precision*recall/(precision+recall)) if ((precision+recall)!=0) else 0)
                tp = tn = fp = fn = 0
            outputs = model(inputs.to(device))
            _, predicted = torch.max(outputs.cpu(), dim=1)
            labels = labels.cpu()
            tp += torch.sum((labels.reshape(-1) == 0) & (predicted == 0)).item()
            tn += torch.sum((labels.reshape(-1) == 1) & (predicted == 1)).item()
            fp += torch.sum((labels.reshape(-1) == 1) & (predicted == 0)).item()
            fn += torch.sum((labels.reshape(-1) == 0) & (predicted == 1)).item()

KeyboardInterrupt: 

In [ ]:
plt.plot(range(len(losses)), losses)

In [ ]:
lens = len(train_accuracies)+1
plt.plot(range(1, lens), train_accuracies, color='red', linestyle='--', label='accuracies')
plt.plot(range(1, lens), train_precisions, color='blue', linestyle='-.', label='precisions')
plt.plot(range(1, lens), train_recalls, color='green', linestyle=':', label='recalls')
plt.plot(range(1, lens), train_F1_scores, color='yellow', linestyle='-', label='F1_scores')
plt.legend()

print(f'训练集精度: {100*train_accuracies[lens-2]}%')

In [ ]:
lens = len(test_accuracies)+1
plt.plot(range(1, lens), test_accuracies, color='red', linestyle='--', label='accuracies')
plt.plot(range(1, lens), test_precisions, color='blue', linestyle='-.', label='precisions')
plt.plot(range(1, lens), test_recalls, color='green', linestyle=':', label='recalls')
plt.plot(range(1, lens), test_F1_scores, color='yellow', linestyle='-', label='F1_scores')
plt.legend()

print(f'测试集准确度: {100*test_accuracies[lens-2]}%')
print(f'测试集精确度: {100*test_precisions[lens-2]}%')
print(f'测试集召回率: {100*test_recalls[lens-2]}%')
print(f'测试集F1值: {100*test_F1_scores[lens-2]}%')

In [ ]:
'''
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs.to(device))
        _, predicted = torch.max(outputs.cpu(), dim=1)
        labels = labels.cpu()
        tp = torch.sum((labels.reshape(-1) == 0) & (predicted == 0)).item()
        tn = torch.sum((labels.reshape(-1) == 1) & (predicted == 1)).item()
        fp = torch.sum((labels.reshape(-1) == 1) & (predicted == 0)).item()
        fn = torch.sum((labels.reshape(-1) == 0) & (predicted == 1)).item()
accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp) if ((tp+fp)!=0) else 0
recall = tp/(tp+fn) if ((tp+fn)!=0) else 0
print(f'测试集准确率: {100*accuracy}%')
print(f'测试集精确率: {100*precision}%')
print(f'测试集召回率: {100*recall}%')
print(f'测试集F1值: {(2*precision*recall/(precision+recall)) if ((precision+recall)!=0) else 0}')
'''

In [ ]:
'''
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs.to(device))
        _, predicted = torch.max(outputs.cpu(), dim=1)
        for i in range(len(predicted)):
            print(f'predict:{predicted[i]}, label:{labels[i]}')
'''

In [ ]:
# torch.save(model.state_dict(), './mymodel/ResNet18/lr=0.0005.pth')